# CutMixRandom

In [ ]:
#| default_exp cutmixrandom

In [ ]:
#| export
# library
from semantic_segmentation_augmentations.holemakertechnique import HoleMakerTechnique
from semantic_segmentation_augmentations.regionmodifier import RegionModifier
from semantic_segmentation_augmentations.iholesfilling import HolesFilling

# others
import random
from fastai.basics import *
import numpy as np

/home/ruescog/.conda/envs/visionmodelsevaluation/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
class CutMixRandom(HolesFilling):
    "Defines the amount of holes, the technique used to make them and the probability of apply the technique."
    def __init__(self,
                 holes_num = 1, # The amount of holes to make.
                 modifier: "RegionModifier" = None, # The modifier that defines the traditional augments to apply to the selected regions.
                 hole_maker: "HoleMakerTechnique" = None, # The strategy used to make the holes.
                 p = 0.5): # The probability of applying this technique.
        super().__init__(modifier, hole_maker)
        self.holes_num = holes_num
        self.p = p

    def before_batch(self):
        "Applies the CutMix technique."
        x, y = tensor(self.x).clone(), tensor(self.y).clone() # tensor is defined in fastai.basics
        for image, mask in zip(self.x, self.y):
            if random.random() < self.p:
                for _ in range(self.holes_num):
                    rand = random.randint(0, self.x.shape[0] - 1)
                    other_image, other_mask = x[rand], y[rand]
                    xhole, yhole = self.make_hole(mask)
                    sub_image, sub_mask = other_image[:, yhole, xhole], other_mask[yhole, xhole]
                    self.fill_hole(image, mask, xhole, yhole, [sub_image, sub_mask])

The default technique used to make those holes is the `HoleMakerRandom` technique.

`CutMix` technique was defined and implemented [here](https://openaccess.thecvf.com/content_ICCV_2019/papers/Yun_CutMix_Regularization_Strategy_to_Train_Strong_Classifiers_With_Localizable_Features_ICCV_2019_paper.pdf).

In [ ]:
show_doc(CutMixRandom.before_batch)

---

### CutMixRandom.before_batch

>      CutMixRandom.before_batch ()

Applies the CutMix technique.

In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()